<a href="https://colab.research.google.com/github/ranamitabh/GenAIPOC/blob/main/Huggingface_Whisper_large_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --q --upgrade pip
!pip install --q --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]
!pip install --q flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 107.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [57]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [58]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [59]:
device

'cuda:0'

In [60]:
torch_dtype

torch.float16

In [61]:
model_id = "openai/whisper-large-v3"

In [62]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [63]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [64]:
pipe

In [80]:
audio1 = "/content/3.mp3"

In [66]:
#%%timeit
#result = pipe(audio) # v2 gives better results if you don't provide a language.

In [67]:
#result = pipe(audio, return_timestamps=True)

In [68]:
#result['text']

In [69]:
#result['chunks']

In [83]:
%%timeit
result = pipe(audio1 , return_timestamps=True, generate_kwargs={"language": "french", "task":"translate"})

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is

2.43 s ± 395 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
result['chunks']


[{'timestamp': (0.0, 2.0), 'text': " C'est un drame fait vers 1920."},
 {'timestamp': (2.0, 4.0), 'text': " Il n'y a pas d'initiative,"},
 {'timestamp': (4.0, 6.0), 'text': ' il faut faire des choses,'},
 {'timestamp': (6.0, 8.0), 'text': ' et il y a des choses qui sont faites'},
 {'timestamp': (8.0, 10.04), 'text': ' dans les comités, dans les réalités,'}]